This notebook will re-analyze the data but only keeping "valid debates" which are ones where NO debaters leak the correct answer during their argument.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '..')
from analysis.analysis_utils import *
from analysis.plot_utils import *
import pandas as pd

In [7]:
all_df = prepare_df()

In [8]:
# Load the debate checks

debate_id = 'q6wpwb7'

file = get_project_root() / 'results' / 'debate_checks' / f'{debate_id}.jsonl'

check_df = pd.read_json(file, lines=True).drop_duplicates(subset=['record_id', 'debater_idx', 'turn_idx'], keep='last')
check_df.shape


(564, 12)

In [10]:
check_df['has_leak'] = check_df['parsed_response'].apply(lambda x: x.get('leaks_detected') if x else None)

record_leak_status = check_df.groupby('record_id')['has_leak'].agg(
    lambda x: False if (x == False).all() else (True if (x == True).any() else None)
).to_dict()

all_df['has_leak_checks'] = all_df['record_id_debates'].map(record_leak_status)
all_df = all_df[all_df['has_leak_checks'] == False]

In [5]:
verdict_ids_2_choice_gpqa =  ['o6nh196', 'se5tc1t', 'okcg9s1', 'px68tl7', 'gt5hbbc', 'omurjjz', '2ayhhpd', '7ir6j2d', 'mr8lipz', 'kn8lecm', 'k26y5y6', 'uopuf76', 'hmsolpd', 'wvubqq5', '0q341gz', 'm6zo8vx', 'jgrph0e']
verdict_ids_4_choice_gpqa =  ['2bmbqk1', 'po3v9oq', 'qx50tbo', 'jm9kod3', '4rg7b4b', 'faqmo5o', 'dph0qre', '9jiz6cv', 'p8zeu0p', 'ouuwuug', 'yt32eoi', 'vwmy4y3', 'qiqr2wo', '2pvnm1z', 'ujzubt9', '4ukh5h7', '3oxc0v3']

df_2_choice_gpqa = all_df[all_df['verdict_run_id_verdicts'].isin(verdict_ids_2_choice_gpqa)]
df_4_choice_gpqa = all_df[all_df['verdict_run_id_verdicts'].isin(verdict_ids_4_choice_gpqa)]

acc_2_choice_gpqa = aggregate_by_fields(df_2_choice_gpqa, ['config_judge_model_verdicts'])
acc_4_choice_gpqa = aggregate_by_fields(df_4_choice_gpqa, ['config_judge_model_verdicts'])

In [ ]:
# Merge the acc_2